In [1]:
def get_pdf_searchable_pages(fname):
    
    # pip install pdfminer
    from pdfminer.pdfpage import PDFPage
    source_dir = 'C:/ML Training Docs/New folder/data/Moving scanned copy/.PDF/'
    target_dir = 'C:/ML Training Docs/New folder/data/Moving scanned copy/scanned_copy/'
    searchable_pages = []
    non_searchable_pages = []
    page_num = 0
    with open(fname, 'rb') as infile:

        for page in PDFPage.get_pages(infile):
            page_num += 1
            if 'Font' in page.resources.keys():
                searchable_pages.append(page_num)
            else:
                non_searchable_pages.append(page_num)
    if page_num > 0:
        if len(searchable_pages) == 0:
            shutil.move(os.path.join(source_dir, file_name), target_dir)
            print(f"Document '{fname}' has {page_num} page(s). "
                  f"Complete document is non-searchable")
        elif len(non_searchable_pages) == 0:
            print(f"Document '{fname}' has {page_num} page(s). "
                  f"Complete document is searchable")
        else:
#             print(f"searchable_pages : {searchable_pages}")
#             if non_searchable_pages != None:
                
#                 shutil.move(os.path.join(source_dir, file_name), target_dir)
            
                print(f"non_searchable_pages : {non_searchable_pages}")
    else:
        print(f"Not a valid document")

In [2]:
import os
import shutil
source_dir = os.path.join('C:/ML Training Docs/New folder/data/Moving scanned copy/.PDF/')

for filename in os.listdir(source_dir):
    file, extension = os.path.splitext(filename)
    if filename[-3:] == 'pdf':
        print(filename)
        
        file_name = source_dir+filename
        get_pdf_searchable_pages(file_name)

1580029.pdf
Document 'C:/ML Training Docs/New folder/data/Moving scanned copy/.PDF/1580029.pdf' has 2 page(s). Complete document is non-searchable
1580047.pdf
Document 'C:/ML Training Docs/New folder/data/Moving scanned copy/.PDF/1580047.pdf' has 1 page(s). Complete document is non-searchable
1580049.pdf
Document 'C:/ML Training Docs/New folder/data/Moving scanned copy/.PDF/1580049.pdf' has 11 page(s). Complete document is non-searchable
1580050.pdf
Document 'C:/ML Training Docs/New folder/data/Moving scanned copy/.PDF/1580050.pdf' has 20 page(s). Complete document is searchable
1580052.pdf
Document 'C:/ML Training Docs/New folder/data/Moving scanned copy/.PDF/1580052.pdf' has 21 page(s). Complete document is non-searchable
1580053.pdf
Document 'C:/ML Training Docs/New folder/data/Moving scanned copy/.PDF/1580053.pdf' has 21 page(s). Complete document is non-searchable
1880052.pdf
non_searchable_pages : [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50

Document 'C:/ML Training Docs/New folder/data/Moving scanned copy/.PDF/7690026.pdf' has 6 page(s). Complete document is searchable
7740029.pdf
Document 'C:/ML Training Docs/New folder/data/Moving scanned copy/.PDF/7740029.pdf' has 6 page(s). Complete document is searchable
7880002.pdf
Document 'C:/ML Training Docs/New folder/data/Moving scanned copy/.PDF/7880002.pdf' has 2 page(s). Complete document is non-searchable
7880032.pdf
Document 'C:/ML Training Docs/New folder/data/Moving scanned copy/.PDF/7880032.pdf' has 98 page(s). Complete document is searchable
7890049.pdf


PDFTextExtractionNotAllowed: Text extraction is not allowed: <_io.BufferedReader name='C:/ML Training Docs/New folder/data/Moving scanned copy/.PDF/7890049.pdf'>

# Docuent classification with splitted data

In [1]:
import nltk
import re
#nltk.download('stopwords')
import pickle
#from nltk.corpus import stopwords

In [2]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [3]:
from sklearn.datasets import load_files
my_data=load_files(r'C:\ML Training Docs\New folder\split\Train')
X,y = my_data.data, my_data.target

In [4]:
import pandas as pd
import numpy as np
df=pd.DataFrame({'filenames':[my_data.filenames],'target':[my_data.target]})
df=explode(df,['filenames','target'],fill_value='',preserve_index=True)
df['target']=df['target'].replace([0,1,2,3,4],['3rd Party Expert','GMC','Solicitor Contacts','MDU Invoices','NHSLA'])
# df.to_csv(r'C:\ML Training Docs\New folder\data_final\data_final.csv',index=False)
df.head(2)

,filenames,target
0,C:\ML Training Docs\New folder\split\Train\MDU...,Solicitor Contacts
0,C:\ML Training Docs\New folder\split\Train\MDU...,Solicitor Contacts


In [5]:

import re
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    
    #Remove \n from text
    document = re.sub(r'\\n', ' ', str(X[sen]))
    
    # remove \t from text
    document = re.sub(r'\\t', ' ', document)
    
     # Remove \\ from the text
    document = re.sub(r'\\', ' ', document)
    
     # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    
    #Remove special character
    document = re.sub(r"[^a-zA-Z0-9]+", ' ', document)
    
   

    
    # Remove one char joins with 2 digit number
    document= re.sub(r'[A-Za-z]{1}[0-9]{2}', ' ', document)
    #Remove 1 digit number joins with 2 char
    document = re.sub(r'[0-9]{1}[A-Za-z]{2}',' ',document)
     #Remove one char in between two numbers
    document = re.sub(r'[0-9]{1}[A-Za-z]{1}[0-9]{1}',' ',document)
    #Remove one number in between two char
    document = re.sub(r'[A-Za-z]{1}[0-9]{1}[A-Za-z]{1}',' ',document)
    
    # Remove two char joins with 2 digit number
    document= re.sub(r'[A-Za-z]{2}[0-9]{2}', ' ', document)
    
    # Remove two char joins with 1 digit number
    document= re.sub(r'[A-Za-z]{2}[0-9]{1}', ' ', document)
    
   
    # Remove double digit number
    document=re.sub('(\\b[0-9][0-9] \\b|\\b [0-9][0-9]\\b)', ' ', document).strip()
    #Remove single character
    document=re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', ' ', document).strip()
    # Remove single digit number
    document=re.sub('(\\b[0-9] \\b|\\b [0-9]\\b)', ' ', document).strip()

    # Substituting multiple spaces with single space
#     document = re.sub(r' +', ' ', document)

    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
#     document = ' '.join(e for e in document if e.isalnum())
    document = ' '.join([w for w in document.split() if len(w)>1])
    
    
    
    documents.append(document)

In [6]:
doc_list = documents
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))

clean_document = [' '.join([b for b in a.split() if b not in stop_words]) for a in doc_list]

In [7]:
import unidecode
accented=[]
for i in range(0,len(clean_document)):
    ach=unidecode.unidecode(clean_document[i])
    accented.append(ach)

In [8]:
accented[0:1]

['young fiona pthl o0 o2 sent subject attachment phillips mark october 2011 ddu advisory fw mr pihl health review pihl claes report gdc sept doc dewhurst charles sent september 2011 phillips mark subject fw mr pihl health review one another complimentary one looking good harry stewart tmaiito harry stewart capsticks com sent september 2011 dewhurst charles cc nimi bruce subject mr pihl health review dear mr dewhurst please find attached psychiatric report dr gilvarry dated september 2011 received email today please note yet received enclosure report informed hard copy ha sent post aim provide copy enclosure soon arrives capsticks kind regard harry stewart paralegal dd 020 8780 4854 harrv stewart caosticks com www capsticks com laing buisson independent healthcare award 2011 winner legal adviser year intended recipient email entitled mr pihl health review please delete email information contained communication confidential may legally privileged intended solely use intended recipient ot

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = vectorizer.fit_transform(accented).toarray()

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,shuffle=True)

In [12]:
'''training the data in RF model'''
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
classifier=RandomForestClassifier()
start = time.time()
classifier.fit(X_train,y_train)
# the time would be round to 3 decimal in seconds
end = time.time()
print ("Random_forest_training time:", round(end-start, 3), "s")
# t1=time.time()
y_pred=classifier.predict(X_test)
# print ("predict time:", round(end-t1, 3), "s")
print('RandomForest:',accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print("Accuracy score (training): {0:.3f}".format(classifier.score(X_train, y_train)))
print("Accuracy score (test): {0:.3f}".format(classifier.score(X_test, y_test)))

Random_forest_training time: 1.167 s
RandomForest: 0.8768472906403941
[[78  1  0  0  6]
 [ 6 62  0  0  4]
 [ 5  9 61  2  7]
 [ 0  0  2 86  0]
 [ 3  1  4  0 69]]
              precision    recall  f1-score   support

           0       0.85      0.92      0.88        85
           1       0.85      0.86      0.86        72
           2       0.91      0.73      0.81        84
           3       0.98      0.98      0.98        88
           4       0.80      0.90      0.85        77

    accuracy                           0.88       406
   macro avg       0.88      0.88      0.87       406
weighted avg       0.88      0.88      0.88       406

Accuracy score (training): 0.998
Accuracy score (test): 0.877


In [13]:
from sklearn.ensemble import GradientBoostingClassifier
classifier=GradientBoostingClassifier()
start = time.time()
classifier.fit(X_train,y_train)
end = time.time()
print ("Gradient_Boost_training time:", round(end-start, 3), "s")
y_pred=classifier.predict(X_test)
print('GradientBoostingClassifier:',accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print("Accuracy score (training): {0:.3f}".format(classifier.score(X_train, y_train)))
print("Accuracy score (test): {0:.3f}".format(classifier.score(X_test, y_test)))

Gradient_Boost_training time: 95.535 s
GradientBoostingClassifier: 0.8817733990147784
[[74  2  1  0  8]
 [ 6 61  1  0  4]
 [ 6  4 68  2  4]
 [ 0  0  3 85  0]
 [ 4  0  3  0 70]]
              precision    recall  f1-score   support

           0       0.82      0.87      0.85        85
           1       0.91      0.85      0.88        72
           2       0.89      0.81      0.85        84
           3       0.98      0.97      0.97        88
           4       0.81      0.91      0.86        77

    accuracy                           0.88       406
   macro avg       0.88      0.88      0.88       406
weighted avg       0.88      0.88      0.88       406

Accuracy score (training): 0.997
Accuracy score (test): 0.882


In [14]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
model.fit(X_train, y_train)
y_pred_lgbm = model.predict(X_test)
print('LGBMClassifier:',accuracy_score(y_test,y_pred_lgbm))
print(confusion_matrix(y_test,y_pred_lgbm))
print(classification_report(y_test,y_pred_lgbm))
print("Accuracy score (training): {0:.3f}".format(classifier.score(X_train, y_train)))
print("Accuracy score (test): {0:.3f}".format(classifier.score(X_test, y_test)))

LGBMClassifier: 0.8669950738916257
[[77  2  1  0  5]
 [ 7 61  0  0  4]
 [ 6  7 64  2  5]
 [ 0  0  1 87  0]
 [ 6  1  7  0 63]]
              precision    recall  f1-score   support

           0       0.80      0.91      0.85        85
           1       0.86      0.85      0.85        72
           2       0.88      0.76      0.82        84
           3       0.98      0.99      0.98        88
           4       0.82      0.82      0.82        77

    accuracy                           0.87       406
   macro avg       0.87      0.86      0.86       406
weighted avg       0.87      0.87      0.87       406

Accuracy score (training): 0.997
Accuracy score (test): 0.882
